In [1]:
from llama_index import SimpleDirectoryReader
import arxiv


Get PDFs

In [2]:
client = arxiv.Client()
search = arxiv.Search(
    query="attention is all you need",
    max_results=5
)




In [3]:
res = client.results(search)

In [4]:
res = list(res)

In [5]:
import os
os.makedirs('./pdfs/', exist_ok=True)
for rs in res:
    rs.download_pdf('./pdfs/', filename=rs.entry_id.split('/')[-1]+'.pdf')
    # rs.download_source('./pdfs/', filename=rs.entry_id.split('/')[-1]+'.tar.gz')

Naive RAG

In [ ]:
import chromadb
from typing import Literal, List
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain.text_splitter import SpacyTextSplitter
import chainlit as cl
import time


In [53]:
from bs4 import BeautifulSoup #type: ignore
from dataclasses import dataclass
from dotenv import load_dotenv
from grobid_client.grobid_client import GrobidClient #type: ignore
from langchain.schema.document import Document
from langchain.text_splitter import SpacyTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.vectorstores import VectorStore
from typing import  Literal
import arxiv #type: ignore
import chainlit as cl
import chromadb
import os
import re
import time
from typing import List
import logging

logging.getLogger("IndexNewArxivPapers").setLevel(logging.INFO)
load_dotenv()


def read_tei(tei_file):
    with open(tei_file, 'r', encoding='utf-8') as tei:
        soup = BeautifulSoup(tei, 'lxml')
        return soup
    raise RuntimeError('Cannot generate a soup from the input')

def elem_to_text(elem, default=''):
    if elem:
        return elem.getText()
    else:
        return default

class TEIFile(object):
    def __init__(self, filename):
        self.filename = filename
        self.soup = read_tei(filename)
        self._text = None
        self._title = ''
        self._abstract = ''

    @property
    def doi(self):
        idno_elem = self.soup.find('idno', type='DOI')
        if not idno_elem:
            return ''
        else:
            return idno_elem.getText()

    @property
    def title(self):
        if not self._title:
            self._title = self.soup.title.getText() #type: ignore
        return self._title

    @property
    def abstract(self):
        if not self._abstract:
            abstract = self.soup.abstract.getText(separator=' ', strip=True) #type: ignore
            self._abstract = abstract
        return self._abstract

    @property
    def authors(self):
        authors_in_header = self.soup.analytic.find_all('author') #type: ignore

        result = []
        for author in authors_in_header:
            persname = author.persname
            if not persname:
                continue
            firstname = elem_to_text(persname.find("forename", type="first")).strip()
            middlename = elem_to_text(persname.find("forename", type="middle")).strip()
            surname = elem_to_text(persname.surname).strip()
            if middlename == '':
                full_name = f"{firstname} {surname}".strip()
            else:
                full_name = f"{firstname} {middlename} {surname}".strip()
            result.append(full_name)
        return result
    
    @property
    def text(self):
        if not self._text:
            divs_text = []
            for div in self.soup.body.find_all("div"): #type: ignore
                # div is neither an appendix nor references, just plain text.
                # if not div.get("type"):
                #     text = div.getText(separator=': ', strip=True).replace("\n", "")
                    
                #     divs_text.append(text)
                print(div)
            plain_text = "\n\n".join(divs_text)
            self._text = plain_text
        return self._text

2024-01-22 23:02:38 - Loaded .env file


In [15]:
from grobid_client.grobid_client import GrobidClient
import os

client = GrobidClient(config_path="./grobid_client_python/config.json")

os.makedirs("./output/", exist_ok=True)
client.process(
    "processFulltextDocument",
    input_path = "./pdfs/",
    output="./output/",
    force=True,
    consolidate_citations=True,
    include_raw_citations=True,
    tei_coordinates=True,
    )



GROBID server is up and running


In [8]:
import json
import grobid_tei_xml

xml_path = "./output/2107.08000v1.grobid.tei.xml"

with open(xml_path, 'r') as xml_file:
    doc = grobid_tei_xml.parse_document_xml(xml_file.read())

with open('./output/2107.08000v1.grobid.tei.json', 'w') as json_file:
    json_file.write(json.dumps(doc.to_dict(), indent=2))

In [27]:
coordinates_list = "6,50.11,649.64,88.59,9.85;6,50.11,668.26,236.25,9.03;6,50.11,680.60,236.25,8.64"

new_coords_list = []
for coordinates in coordinates_list.split(';'):
    page_num = int(coordinates.split(',')[0])
    coordinates = [float(x) for x in coordinates.split(',')[1:]]
    new_coords_list.append((page_num, coordinates))
print(new_coords_list) 


In [31]:
from PyPDF2 import PdfReader, PdfWriter
from pdf2image import convert_from_path
import os
def crop_pdf(input_pdf, output_pdf, coordinates):
    # Create a PDF file reader
    reader = PdfReader(input_pdf)
    page_num = coordinates[0]
    coords = coordinates[1]

    # Create a PDF file writer
    writer = PdfWriter()

    # For each page, crop it and add it to the new PDF
    page = reader.pages[page_num - 1]
    
    page_x, page_y = page.cropbox.get()

    # Write the output PDF
    with open(output_pdf, "wb") as out_f:
        writer.write(out_f)

    # Convert the cropped PDF to PNG
    images = convert_from_path(output_pdf)

    # Save the first page as a PNG
    images[0].save('output.png', 'PNG')
    os.remove(output_pdf)

# Parse the coordinates

# Crop the PDF
crop_pdf("pdfs/2107.08000v1.pdf", "output.pdf", coords)



[(6, [50.11, 649.64, 88.59, 9.85]), (6, [50.11, 668.26, 236.25, 9.03]), (6, [50.11, 680.6, 236.25, 8.64])]


In [10]:
print(doc.body)

Introduction Instance-level image retrieval is at the core of visual representation learning and is connected with many problems of visual recognition and machine learning, for instance metric learning  [30, 26] , few-shot learning  [42]  and unsupervised learning  [8] . Many large-scale open datasets  [3, 37, 16, 29, 53] , and competitions 1 have accelerated progress in instance-level image retrieval, which has been transformed by deep learning  [3] . Many studies on instance-level image retrieval focus on learning features from convolutional neural networks (CNN), while others focus on re-ranking, for instance by graph-based methods  [11] . The former can be distinguished according to feature types: local descriptors, reminiscent of SIFT  [27] , where an image is mapped to a few hundred vectors; and global descriptors, where an image is mapped to a 1 https://www.kaggle.com/c/landmark-retrieval-2020 single vector. In fact, deep learning has brought global descriptors with astounding p

In [ ]:
with open("./output/2306.01926v1.grobid.tei.xml", "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "lxml-xml")
chunks = []
content = ""
for div in soup.body.find_all(["div", "figure"]):
    if div.name == "figure":
        head = div.find("head")
        fig_desc = div.find("figDesc")
        # if ('Figure' not in head.getText()) and ('Table' not in head.getText()):
        content += head.getText().replace(" ", "") + fig_desc.getText() + "\n"
    elif div.name == "div":
        section_num = div.head.get("n")
        section_title = div.head.getText(separator=' ', strip=True)
        if section_num is None:
            section_num = ""
        content += section_num + " " + div.getText(separator=': ', strip=True) + "\n"
    content += "=======================================\n"
    # for head in div.find_all("head"):
    #     section_num = head.get("n")
    #     section_title = head.getText(separator=' ', strip=True)
    #     print(section_num, section_title)
    # content = ""
    
    # for p in div.find_all("p"):
    #     content += p.getText(separator=' ', strip=True)
    # print(content)
    # print("=================================")
    

In [86]:
with open("output.txt", "w", encoding="utf-8") as f:
    f.write(content)

In [6]:

class IndexNewArxivPapers:
    """
    This tool indexes new papers from arxiv using the following steps:
    
    1. Get the paper ids from google search
    2. Download the papers
    3. Process the papers with GROBID
    4. Parse the TEI files
    5. Add the papers to the vectorstore
    
    """
    def __init__(self, 
                vectordb: VectorStore,
                n_search_results: int = 2,
                pdf_parser: Literal["grobid", "pymupdf"] = "grobid",
                chunk_size: int = 1024,
                chunk_overlap: int = 100,
                ):
    
        self.google_api = GoogleSearchAPIWrapper()
        self.arxiv_client = arxiv.Client(delay_seconds=0)
        self.vectordb = vectordb
        self.chromadb_client = chromadb.PersistentClient("arxiv_vdb").get_collection("arxiv")
        self.n_search_results = n_search_results
        self.splitter = SpacyTextSplitter(
            chunk_size = chunk_size,
            chunk_overlap = chunk_overlap,
            separator="\n\n")
        self.pdf_parser = pdf_parser
    
    def _get_paper_ids(self, query: str) -> List[str]:
        ARXIV_ID_REGEX =  r"\d{4}\.\d{4,5}"
        
        ids = list(
            {re.findall(ARXIV_ID_REGEX, result['link'])[0] 
             for result in self.google_api.results(query, self.n_search_results)}
            )
        
        return ids
    
    def _run(self, query):
        with cl.Step():
            self.ids = self._get_paper_ids(query)
            
            os.makedirs(f"./output/{query}", exist_ok=True)
            os.makedirs(f"./pdfs/{query}", exist_ok=True)
            
            for id in self.ids:
                if len(self.chromadb_client.get(where={'paper_id': id})['ids']) > 0:
                    self.ids.remove(id)
                else:
                    continue
            
            papers = list(self.arxiv_client.results(arxiv.Search(id_list=self.ids)))
            for paper in papers:
                id = paper.entry_id.split('/')[-1]
                paper.download_pdf(dirpath=f"./pdfs/{query}/", filename=f"{paper.entry_id.split('/')[-1]}.pdf")
            
            while not all([os.path.exists(f"./pdfs/{query}/{paper.entry_id.split('/')[-1]}.pdf") for paper in papers]):
                time.sleep(1)
            
            docs = []
            if self.pdf_parser == "grobid":
                self.grobid_client = GrobidClient(config_path="./grobid_client_python/config.json")
                self.grobid_client.process("processFulltextDocument", f"./pdfs/{query}/", output=f"./output/{query}", force=True)            
                
                while not all([os.path.exists(f"./output/{query}/{paper.entry_id.split('/')[-1]}.grobid.tei.xml") for paper in papers]):
                    time.sleep(1)
            
                for paper in papers:
                    tei_object = TEIFile(f"./output/{query}/{paper.entry_id.split('/')[-1]}.grobid.tei.xml")
                    chunks = self.splitter.split_text(tei_object.text)
                    docs.extend([
                            Document(
                                    page_content=chunk, 
                                    metadata={
                                        'paper_id': paper.entry_id.split("/")[-1],
                                        'authors': ", ".join([author.name for author in paper.authors]),
                                        'title': paper.title,
                                        'chunk_id': f"{paper.entry_id.split('/')[-1]}-{i}"
                                        }
                                    )
                                    for i, chunk in enumerate(chunks)
                                ]
                    )
                    
            if self.pdf_parser == "pymupdf":
                for paper in papers:
                    loader = PyMuPDFLoader(f"pdfs/{query}/{paper.entry_id.split('/')[-1]}.pdf")
                    chunks = loader.load_and_split(self.splitter)                    
                    docs.extend([
                            Document(
                                    page_content=chunk.page_content, 
                                    metadata={
                                        'paper_id': paper.entry_id.split("/")[-1],
                                        'authors': ", ".join([author.name for author in paper.authors]),
                                        'title': paper.title,
                                        'chunk_id': f"{paper.entry_id.split('/')[-1]}-{i}"
                                        }
                                    ) 
                                    for i, chunk in enumerate(chunks)
                                ]
                        )
            self.vectordb.add_documents(
                docs
            )


    async def _arun(self, query: str):
        async with cl.Step():
            self._run(query)

GROBID server is up and running


In [4]:
from arxiv_bot.search import TEIFile
from llama_index.vector_stores import ChromaVectorStore
from llama_index import Document, VectorStoreIndex
from langchain.text_splitter import SpacyTextSplitter

splitter = SpacyTextSplitter(chunk_size=1024, chunk_overlap=100, separator="\n\n")
for paper in os.listdir("./output/"):
    tei_object = TEIFile(f"./output/{paper}")
    print(tei_object.text)
    print("===================")



The key to a Transformer model is the self-attention mechanism, which allows the model to analyze an entire sequence in a computationally efficient manner. Recent work has suggested the possibility that general attention mechanisms used by RNNs could be replaced by active-memory mechanisms. In this work, we evaluate whether various activememory mechanisms could replace self-attention in a Transformer. Our experiments suggest that active-memory alone achieves comparable results to the self-attention mechanism for language modelling, but optimal results are mostly achieved by using both active-memory and self-attention mechanisms together. We also note that, for some specific algorithmic tasks, active-memory mechanisms alone outperform both the self attention and a combination of the two.

Introduction: The previous state-of-the-art sequence model, the recurrent neural network, has been largely supplanted by the Transformer model: [Vaswani et al., 2017]: , which is primarily built atop a

c:\Users\muraf\Courses\arxiv_llm\venv\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [64]:
docs.load_data()

[Document(id_='5890981b-8362-4a80-9cd6-7cf661effa8b', embedding=None, metadata={'page_label': '1', 'file_name': '1912.11959v2.pdf', 'file_path': 'pdfs\\1912.11959v2.pdf', 'file_type': 'application/pdf', 'file_size': 399865, 'creation_date': '2024-01-19', 'last_modified_date': '2024-01-19', 'last_accessed_date': '2024-01-19'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, hash='a970d4d7e2cfc588d7ecbb820b81062b422baf9760e2fc006ef01d191ed6009d', text='Is Attention All What You Need ? - An Empirical Investigation on\nConvolution-Based Active Memory and Self-Attention\nThomas Dowdell ,Hongyu Zhang\nThe University of Newcastle, NSW, Australia\ntomjamesdowdell@gmail.com, hongyu.zhang@newcastle.edu.au\nAbstract\nThe key to a Transformer model is the self-attention\n